In [57]:
import urllib3
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

def webscrap_single( num = 0 ):
    http = "http://www.policja.pl/pol/form/1,Statystyki-dnia.html?page=%d" % num
    r = urllib3.PoolManager().request( 'GET', http )
    soup = BeautifulSoup( r.data )
    
    cells = soup.find_all("td")
    
    arr = [ [] for i in range(7) ]
    
    count = 0
    for cell in cells:
        text = cell.find(text=True)
        
        if text.find("-") != -1:
            break
        
        arr[count % 7].append( text )
        count+=1
    
    arr[0] = np.array( [ list(map( int, date.split(".") )) for date in arr[0] ] )
    
    
    df = pd.DataFrame()
    # day month year intercepted fugitive drunk accidents killed injured
    
    df["year"]  = arr[0][:,2]
    df["month"] = arr[0][:,1]
    df["day"]   = arr[0][:,0]
    
    df["intercepted"] = np.array( arr[1] )
    df["fugitive"]    = np.array( arr[2] )
    df["drunk"]       = np.array( arr[3] )
    df["accidents"]   = np.array( arr[4] )
    df["killed"]      = np.array( arr[5] )
    df["injured"]     = np.array( arr[6] )
    
    return df

In [56]:
webscrap_single(10)

,year,month,day,intercepted,fugitive,drunk,accidents,killed,injured
0,2016,12,11,634,139,280,79,5,103
1,2016,12,10,682,163,276,103,13,114
2,2016,12,9,584,295,213,102,9,99
3,2016,12,8,503,287,121,77,6,98
4,2016,12,7,468,273,126,90,7,101
5,2016,12,6,490,274,156,95,6,121
6,2016,12,5,490,232,174,84,8,95
7,2016,12,4,593,150,226,54,9,63
8,2016,12,3,564,170,346,101,13,128
9,2016,12,2,488,241,139,117,11,123


In [75]:
def webscrap_all( t1 = 0, t2 = 204 ):
    dfs = []
    
    for t in range( t1, t2 ):
        dfs.append( webscrap_single( t ) )
    
    return pd.concat( dfs ).sort_values(["year", "month", "day"]).reset_index(drop=True)

In [76]:
df = webscrap_all()
df.to_csv("data.csv", index = False)